# Data Processing Notebook (Python)
Project: GEOM90007 Assignment 3 – Group Project\n\n
This notebook reads all CSV files under `data/raw/`, applies dataset-specific filters,\n
and writes analysis-ready outputs to `data/processed/`.

In [13]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', 120)
print('pandas', pd.__version__)


pandas 2.2.2


In [15]:
# Paths (robust to running from repo root or data/)
ROOT = Path('.').resolve()
candidates = [
    ROOT / 'data',           # notebook at repo root
    ROOT,                    # notebook already inside data/
    ROOT.parent / 'data'     # fallback: run from subfolder
]
DATA = None
for c in candidates:
    if (c / 'raw').exists():
        DATA = c
        break
if DATA is None:
    raise FileNotFoundError(f"Could not locate 'data/raw' from {ROOT}")

RAW = DATA / 'raw'
PROCESSED = DATA / 'processed'

# Do not create RAW; it must already exist in the repo.
if not (RAW.exists() and RAW.is_dir()):
    raise FileNotFoundError(f"Expected existing raw folder at {RAW}")
# Ensure only output folders exist
PROCESSED.mkdir(parents=True, exist_ok=True)

RAW, PROCESSED


(PosixPath('/Users/wangwang/Desktop/School/GEO90007/GEOM90007_Assignment3_Group_Project/data/raw'),
 PosixPath('/Users/wangwang/Desktop/School/GEO90007/GEOM90007_Assignment3_Group_Project/data/processed'))

In [ ]:
# Helpers
def snake(s: str) -> str:
    s = re.sub(r'[^0-9a-zA-Z]+', '_', s.strip())
    s = re.sub(r'_+', '_', s)
    return s.strip('_').lower()

def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    return df.rename(columns=lambda c: snake(str(c)))

def read_csv_any(path: Path) -> pd.DataFrame:
    try:
        return pd.read_csv(path)
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding='latin-1')

def find_col(df: pd.DataFrame, *keys, default=None):
    cols = {c.lower(): c for c in df.columns}
    for k in keys:
        for name, orig in cols.items():
            if k in name:
                return orig
    return default

def ensure_datetime(df: pd.DataFrame, prefer_month=True):
    # Try common datetime columns
    cands = ['date_time','datetime','timestamp','date','period','month','time']
    for c in cands:
        col = find_col(df, c)
        if col is not None:
            dt = pd.to_datetime(df[col], errors='coerce')
            if dt.notna().any():
                return dt
    # Try year/month/day combos
    y = find_col(df, 'year')
    m = find_col(df, 'month')
    d = find_col(df, 'day')
    if y is not None and m is not None:
        day = df[d] if d else 1
        try:
            return pd.to_datetime(dict(year=df[y], month=df[m], day=day), errors='coerce')
        except Exception:
            pass
    return pd.Series(pd.NaT, index=df.index)

def between_dates(df: pd.DataFrame, start, end) -> pd.DataFrame:
    dt = ensure_datetime(df)
    mask = (dt >= pd.to_datetime(start)) & (dt <= pd.to_datetime(end))
    out = df.loc[mask].copy()
    out['__dt'] = dt[mask]
    return out

def write_out(df: pd.DataFrame, name: str):
    path = PROCESSED / name
    df.to_csv(path, index=False)
    print('Wrote', path)


In [ ]:
# Discover raw files
print('Using RAW folder:', RAW)
raw_files = {f.name: f for f in sorted(RAW.glob('*.csv'))}
print('Found CSV files:', len(raw_files))
list(raw_files.keys())


## Time Window
We will repeatedly use this window for filtering: 2024-10 to 2025-10.

In [ ]:
START = '2024-10-01'
END   = '2025-10-31'
START, END


## 1) exchange_rates_monthly.csv
- Keep months 2024-10 .. 2025-10 inclusive.
- Keep currencies: CNY, NZD, INR, SGD, HKD, MYR, JPY, USD, GBP, IDR.
- Output tidy columns: date, currency, rate.

In [59]:
ex_path = raw_files.get('exchange_rates_monthly.csv')

ex = clean_columns(read_csv_any(ex_path))
currencies = ['units', 'usd','cny','nzd','inr','sgd','hkd','myr','jpy','gbp','idr']
ex = ex[currencies]
write_out(ex, 'exchange_rates_2024_9_to_2025_9.csv')


Wrote /Users/wangwang/Desktop/School/GEO90007/GEOM90007_Assignment3_Group_Project/data/processed/exchange_rates_2024_9_to_2025_9.csv


## 2) fares_for_bi.csv
- Keep 2020–2025 and rows related to Melbourne.

In [ ]:
fares_path = raw_files.get('fares_for_bi.csv')

fr = clean_columns(read_csv_any(fares_path))

# Keep years 2020–2025
fr_filtered = fr[(fr['year'] >= 2020) & (fr['year'] <= 2025)].copy()

# Keep rows containing "melbourne" in any text column
text_cols = [c for c in fr_filtered.columns if fr_filtered[c].dtype == 'object']
melb_mask = pd.Series(False, index=fr_filtered.index)
for c in text_cols:
    melb_mask |= fr_filtered[c].astype(str).str.contains('melbourne', case=False, na=False)

fr_melb = fr_filtered[melb_mask].copy()
write_out(fr_melb, 'fares_for_bi_2020_2025_melbourne.csv')


Wrote /Users/wangwang/Desktop/School/GEO90007/GEOM90007_Assignment3_Group_Project/data/processed/fares_for_bi_2020_2025_melbourne.csv


## 3) landmarks_and_places_...csv
- Keep only selected Sub Theme categories (fuzzy matching).

In [48]:
lm_name = 'landmarks_and_places_of_interest_including_schools_theatres_health_services_spor.csv'
lm_path = raw_files.get(lm_name)

targets = [
    'Art Gallery / Museum',
    'Church',
    'Function / Conference / Exhibition Centre',
    'Informal Outdoor Facility (Park / Garden / Reserve)',
    'Major Sports & Recreation Facility',
    'Theatre / Live Entertainment Venue',
    'Tertiary (University)'
]


lm = clean_columns(read_csv_any(lm_path))

if 'sub_theme' in lm.columns:
    lm_selected = lm[lm['sub_theme'].isin(targets)].copy()
    write_out(lm_selected, 'landmarks_selected_subthemes.csv')
else:
    print('sub_theme column not found — writing cleaned copy only.')
    write_out(lm, 'landmarks_clean.csv')



Wrote /Users/wangwang/Desktop/School/GEO90007/GEOM90007_Assignment3_Group_Project/data/processed/landmarks_selected_subthemes.csv


## 4) pedestrian_counting_system_monthly_counts_per_hour.csv
- Keep 2024-10 .. 2025-10.

In [ ]:
ped_path = raw_files.get('pedestrian_counting_system_monthly_counts_per_hour.csv')
ped = clean_columns(read_csv_any(ped_path))

# Parse sensing_date directly
ped['__dt'] = pd.to_datetime(ped['sensing_date'], errors='coerce')

# Filter by date range
ped_f = ped[(ped['__dt'] >= pd.to_datetime(START)) &
            (ped['__dt'] <= pd.to_datetime(END))].copy()

write_out(ped_f.drop(columns=['__dt']), 'pedestrian_counts_2024_10_to_2025_10.csv')


Wrote /Users/wangwang/Desktop/School/GEO90007/GEOM90007_Assignment3_Group_Project/data/processed/pedestrian_counts_2024_10_to_2025_10.csv


## 5) weather_microclimate_sensors_data.csv
- Keep 2024-10 .. 2025-10.

In [ ]:
wx_path = raw_files.get('weather_microclimate_sensors_data.csv')

wx = clean_columns(read_csv_any(wx_path))
sensors = ['aws5-0999', 'ICTMicroclimate-11', 'ICTMicroclimate-09', 'ICTMicroclimate-07']
wx = wx[wx["device_id"].astype(str).str.strip().isin(sensors)].copy()

# Step 1: convert time to Melbourne TZ and drop timezone
wx['Target_Time'] = (
    pd.to_datetime(wx['time'], utc=True)
      .dt.tz_convert('Australia/Melbourne')
      .dt.tz_localize(None)
)

# Step 2: filter by date window
wx = wx[(wx['Target_Time'] >= pd.to_datetime(START)) &
        (wx['Target_Time'] <= pd.to_datetime(END))].copy()

# Step 3: aggregate by date
wx['Target_Date'] = wx['Target_Time'].dt.date
agg_cols = [
    'minimumwinddirection', 'averagewinddirection', 'maximumwinddirection',
    'minimumwindspeed', 'averagewindspeed', 'gustwindspeed',
    'airtemperature', 'relativehumidity', 'atmosphericpressure',
    'pm25', 'pm10', 'noise'
]

wx_daily = (
    wx.groupby(['device_id', 'sensorlocation', 'latlong', 'Target_Date'])[agg_cols]
      .mean()
      .reset_index()
)

# Step 4: format as string for output
wx_daily['Target_Time'] = pd.to_datetime(wx_daily['Target_Date']).dt.strftime('%Y-%m-%d %H:%M:%S')
wx_daily = wx_daily.drop(columns=['Target_Date'])

write_out(wx_daily, 'weather_microclimate_daily_avg.csv')


Wrote /Users/wangwang/Desktop/School/GEO90007/GEOM90007_Assignment3_Group_Project/data/processed/weather_microclimate_daily_avg.csv


## 6) short_term_visitor_arrivals,_state_of_stay_2025.csv
- Generic cleaning for analysis (no special filter specified).

In [ ]:
st_name = 'short_term_visitor_arrivals,_state_of_stay_2025.csv'
st_path = raw_files.get(st_name)

st = clean_columns(read_csv_any(st_path))
# Basic tidying: remove empty cols/rows, attempt datetime parse for any date-like col
st = st.dropna(how='all').copy()
# If monthly columns present, try to build a datetime index
dt = ensure_datetime(st)
if dt.notna().any():
    st['date'] = dt
write_out(st, 'short_term_visitor_arrivals_clean.csv')



In [66]:
st_name = 'short_term_visitor_arrivals,_state_of_stay_2019_2025.csv'
st_path = raw_files.get(st_name)

st = clean_columns(read_csv_any(st_path))


# Rename columns
df = st.rename(columns={
    'state_or_territory_of_stay': 'state',
    'aug_2019_no': 'Aug_2019',
    'aug_2023_no': 'Aug_2023',
    'aug_2024_no': 'Aug_2024',
    'aug_2025_no': 'Aug_2025',
    'aug_2024_to_aug_2025_change': 'change_pct'
})

# Remove commas and convert to numeric
num_cols = ['Aug_2019','Aug_2023','Aug_2024','Aug_2025','change_pct']
for c in num_cols:
    df[c] = df[c].astype(str).str.replace(',', '').str.replace('"', '').astype(float)

# Melt to long format
df_long = df.melt(
    id_vars=['state', 'change_pct'],
    value_vars=['Aug_2019','Aug_2023','Aug_2024','Aug_2025'],
    var_name='period',
    value_name='value'
)

# Extract month & year
df_long['month'] = df_long['period'].str.extract(r'([A-Za-z]+)')
df_long['year'] = df_long['period'].str.extract(r'(\d{4})').astype(int)

# Reorder columns
df_long = df_long[['state','year','month','value','change_pct']].sort_values(['state','year'])

write_out(df_long, 'cleaned_short_term_visitor_arrivals,_state_of_stay_2019_2025.csv')



Wrote /Users/wangwang/Desktop/School/GEO90007/GEOM90007_Assignment3_Group_Project/data/processed/cleaned_short_term_visitor_arrivals,_state_of_stay_2019_2025.csv


In [72]:
st_name = 'short_term_visitor_arrivals_Vic_top_10_source_countries.csv'
st_path = raw_files.get(st_name)

st = clean_columns(read_csv_any(st_path))


# Rename columns
df = st.rename(columns={
    'country_of_residence': 'Country',
    'aug_19': 'Aug_2019',
    'aug_23': 'Aug_2023',
    'aug_24': 'Aug_2024',
    'aug_25': 'Aug_2025'})
# Remove commas and convert to numeric
num_cols = ['Aug_2019','Aug_2023','Aug_2024','Aug_2025']
for c in num_cols:
    df[c] = df[c].astype(str).str.replace(',', '').str.replace('"', '').astype(float)


write_out(df, 'cleaned_short_term_visitor_arrivals_Vic_top_10_source_countries.csv')



Wrote /Users/wangwang/Desktop/School/GEO90007/GEOM90007_Assignment3_Group_Project/data/processed/cleaned_short_term_visitor_arrivals_Vic_top_10_source_countries.csv


### Done
Processed datasets are saved under `data/processed/`.